# LDAP Auth Method

Vault is an API driven software component aimed at automation. Thus most authentications to Vault are performed via machines, using authentication methods such as JWT or mTLS. However, Vault needs to be operated and secured by people. Moreover, it is not too uncommon to see Vault used as a "glorified shared password manager" and thus requires human users to authenticate to it.

While other solutions exist that enable SSO (e.g. OIDC or Kerberos), a very common way to authenticate human users to Vault is via LDAP. This is mostly due to the fact that its setup is very straight forward, and typically flexible enough. In this module we will have a look at LDAP authentication.

In case you are not familiar with it, LDAP is a protocol that enables you to communicate with directory servers, such as Microsoft Active Directory (AD). Such directory servers (often also called domain controllers in the Windows world) store information about entities within an organisation. They typically have an internal representation of the organisation's structure, and all its employees within that structure. Domain controllers are used to store much more than human users and org structures, but this is what we will focus on in this module.

Directory servers typically contain user information such as usernames, group membership, and a password. This enables Vault to effectively delegate authentication of a user to such a server (via LDAP). After successful authentication, the group membership can then be used to grant permissions to a user. This is very convenient, as joiner/leaver/mover processes are typically implemented on such servers, meaning that new employees are automatically onboarded to Vault as soon as they are added to some groups within the directory server.

If this is slightly unclear, either have a read on [directory services](https://en.wikipedia.org/wiki/Directory_service) and [LDAP](https://en.wikipedia.org/wiki/Lightweight_Directory_Access_Protocol), or just move on and discover it as we go along.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

For now, do not worry about the configuration with which we are starting the server. This will be covered in a separate module.

In [ ]:
nohup bash -c '
  vault server -dev -dev-root-token-id=root-token -dev-listen-address="0.0.0.0:8200"
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Preparing the LDAP Server

Before we can started with this module, we will need to have a directory service up and running. Let us start one.

In [ ]:
docker run --detach --rm --name openldap \
  --network kind \
  -e LDAP_ADMIN_USERNAME=admin \
  -e LDAP_ADMIN_PASSWORD=admin \
  -e LDAP_ROOT=dc=example,dc=org \
  -e LDAP_ADMIN_DN=cn=admin,dc=example,dc=org \
  -p 1389:1389 \
  bitnami/openldap:2.6.9-debian-12-r16

### Investigating the Structure

The above command started an OpenLDAP directory server with some pre-configured data for us. It configured the following:

- A base domain of `example.org`.
- An admin user with password `admin`, whose distinguished name is `cn=admin,dc=example,dc=org`. These names are essentially a unique reference to a directory element. See more information [here](https://learn.microsoft.com/en-us/previous-versions/windows/desktop/ldap/distinguished-names) if you are interested.
- Two users, `cn=user01,ou=users,dc=example,dc=org` and `cn=user02,ou=users,dc=example,dc=org` with password `bitnami1` and `bitnami2` respectively.
- A group `cn=readers,ou=groups,dc=example,dc=org` which contains both users above.

Have a look at this structure by executing the command below:

> NOTE: a copy of the output can be found under `assets/base.ldif` for reference as well.

In [ ]:
if [ -f /.dockerenv ]; then
    export DIR_SER_ADDR=openldap
else
    export DIR_SER_ADDR="127.0.0.1"
fi
ldapsearch -x -H "ldap://${DIR_SER_ADDR}:1389" -b "dc=example,dc=org"

### Setup Up the AuthN Method

We are now at a point where most operators would start configuring the Vault. The directory servers are typically something that is pre-existing in the organisation. Now in order to configure authentication, we will need to enable the authentication method. We will then need to configure Vault to connect to the directory server located at a specific address and using a so called bind DN (a fancy word for user). On top of that, since directory servers are extremely flexible in terms of how they can be structured internally, we will need to tell Vault where in the structure it can find users, where it can find groups, and then define a mapping between such a group and permissions on the Vault.

Let us first enable the [authentication method](https://developer.hashicorp.com/vault/docs/auth/ldap):

In [ ]:
# login
export VAULT_ADDR="http://127.0.0.1:8200"
vault login root-token
# enable the method
vault auth enable ldap

### Configuring the Connection

We now need to tell Vault where to find the directory server. Since our OpenLDAP server is not secured via a TLS certificate, we will also need to tell Vault to accept this configuration. Generally we will set:

- DiscoverDN: tells Vault to connect anonymously to the LDAP server and find the UserDN that way. This is often allowed in actual LDAP servers.
- URL: the url under which to find the OpenLDAP server.
- Insecure: we do not want to connect via TLS.
- UserDN: where in the directory server structure to look for user objects.
- UserAttr: what attribute of a user object to use as the username.
- GroupDN: where in the directory server structure to look for group objects.
- Group Filter: how to filter which groups a user belongs to. Don't worry if you don't understand this, it is OpenLDAP black magic. It essentially says that a user is part of a group if the UserDN is listed under the `member`s fields of the group.
- GroupAttr: what attribute of a group object to use as the group name.
- Token no default policy: do not assign default policies to tokens that login in via this authentication method by default. This is necessary to ensure that users that are not part of any group do not get a token with default policies.

In [ ]:
vault write auth/ldap/config \
  discoverdn=true \
  url="ldap://${DIR_SER_ADDR}:1389" \
  insecure_tls=true \
  userdn="ou=users,dc=example,dc=org" \
  userattr="cn" \
  groupdn="ou=groups,dc=example,dc=com" \
  groupfilter="(&(objectClass=group)(member:1.2.840.113556.1.4.1941:={{.UserDN}}))" \
  groupattr="cn" \
  token_no_default_policy=true

### Granting Permissions

Before we can attempt a login, we need to ensure that we somehow link a group membership to a set of permissions. This is much simpler than the configuration of the method, do not worry. This is done by creating a LDAP group on the Vault and providing a set of policies. The group in Vault must simply match the name of the group in the LDAP server (in our case `readers`).

Configure the `readers` group to be granted the `read-all` policy when login in. The 

In [ ]:
vault write auth/ldap/groups/<group> \
  policies="<policy>"

### Login

Now let us attempt a login with a user. Note that typically only the first CN can be used as the username if several are defined in the directory server.

In [ ]:
vault login -method=ldap username=<user> password=<password>

### ADVANCED: Adding Users to Groups

You can also add users to groups manually within Vault. This is especially important when users are part of several groups and you want to grant them permissions from several groups, or you want to add a user that is not part of some group on the Directory Server but you still want to authenticate using the LDAP authentication method.

Have a look at the API documentation of the user endpoints: https://developer.hashicorp.com/vault/api-docs/auth/ldap#create-update-ldap-user

Perform the following:

1. Create an additional group on the LDAP server which includes some new users, and one of the existing users.
2. Onboard the new group on Vault providing it with a different policy.
3. Login with the user that is in both groups and check the granted policies.
4. Create a new user that is not a member of any group. Onboard it on the Vault using the LDAP authentication method and login using its credentials.

### Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault.log
rm /tmp/vault.pid
docker stop openldap